In [ ]:
from dotenv import load_dotenv


load_dotenv()

# Imports

In [ ]:
import json
import os

from libdc3.config import dc3_config
from libdc3.methods.acc_lumi_analyzer import AccLuminosityAnalyzer
from libdc3.methods.bril_actions import BrilActions
from libdc3.methods.era_plotter import EraPlotter
from libdc3.methods.json_producer import JsonProducer
from libdc3.methods.lumiloss_analyzer import LumilossAnalyzer
from libdc3.methods.lumiloss_plotter import LumilossPlotter
from libdc3.methods.rr_actions import RunRegistryActions
from libdc3.methods.t0_actions import T0Actions
from libdc3.services.caf.client import CAF

# User input

In [ ]:
output_path = "./results/calls_1_24_complete"

# Call specific numbers
cycles_list = [
    "Collisions2024_certification_call1",
    "Collisions2024_certification_call2",
    "Collisions2024_certification_call3",
    "Collisions2024_certification_call4",
    "Collisions2024_certification_call5",
    "Collisions2024_certification_call6",
    "Collisions2024_certification_call7",
    "Collisions2024_certification_call8",
    "Collisions2024_certification_call9",
    "Collisions2024_certification_call10",
    "Collisions2024_certification_call11",
    "collisions2024_certification_call12",
    "collisions2024_certification_call13",
    "collisions2024_certification_call14",
    "collisions2024_certification_call15",
    "Collisions2024_certification_call16",
    "Collisions2024_certification_call17",
    "Collisions2024_certification_call18",
    "Collisions2024_certification_call19",
    "Collisions2024_certification_call20",
    "Collisions2024_certification_call21",
    "Collisions2024_certification_call22",
    "Collisions2024_certification_call23",
    "Collisions2024_certification_call24",
]
ignore_runs_in_any_era = [379337, 379318, 379317, 379316, 379315, 379350, 379349, 378981]

# Run registry specific variables
rr_class_name = "Collisions24"
rr_dataset_name = "/PromptReco/Collisions2024/DQM"

# Bril specific variables
bril_brilws_version = "3.7.4"
bril_unit = "/ub"
bril_low_lumi_thr = 80000.0
bril_beamstatus = "STABLE BEAMS"
bril_amodetag = "PROTPHYS"
bril_normtag = "/cvmfs/cms-bril.cern.ch/cms-lumi-pog/Normtags/normtag_BRIL.json"

# Json production specific
ignore_hlt_emergency = False
prejson_oms_flags = ["beam1_present", "beam2_present", "beam1_stable", "beam2_stable"]
goldenjson_oms_flags = [
    "beam1_present",
    "beam2_present",
    "beam1_stable",
    "beam2_stable",
    "cms_active",
    "bpix_ready",
    "fpix_ready",
    "tibtid_ready",
    "tecm_ready",
    "tecp_ready",
    "tob_ready",
    "hbhea_ready",
    "hbheb_ready",
    "hbhec_ready",
    "hf_ready",
    "ho_ready",
]
goldenjson_rr_flags = [
    "tracker-pixel",
    "tracker-strip",
    "tracker-track",
    "ecal-ecal",
    "ecal-es",
    "hcal-hcal",
    "csc-csc",
    "dt-dt",
    "l1t-l1tmu",
    "l1t-l1tcalo",
    "hlt-hlt",
    "egamma-egamma",
    "muon-muon",
    "jetmet-jetmet",
]
muonjson_oms_flags = [
    "beam1_present",
    "beam2_present",
    "beam1_stable",
    "beam2_stable",
    "cms_active",
    "bpix_ready",
    "fpix_ready",
    "tibtid_ready",
    "tecm_ready",
    "tecp_ready",
    "tob_ready",
]
muonjson_rr_flags = [
    "tracker-pixel",
    "tracker-strip",
    "tracker-track",
    "csc-csc",
    "dt-dt",
    "l1t-l1tmu",
    "hlt-hlt",
    "muon-muon",
]

# Lumiloss specific
target_lumiloss_unit = "/pb"
lumiloss_dcs_flags = [
    "bpix_ready",
    "fpix_ready",
    "tibtid_ready",
    "tecm_ready",
    "tecp_ready",
    "tob_ready",
    "hbhea_ready",
    "hbheb_ready",
    "hbhec_ready",
    "hf_ready",
    "ho_ready",
]
lumiloss_subsystems_flags = [
    "tracker-pixel",
    "tracker-strip",
    "tracker-track",
    "ecal-ecal",
    "ecal-es",
    "hcal-hcal",
    "csc-csc",
    "dt-dt",
    "l1t-l1tmu",
    "l1t-l1tcalo",
    "hlt-hlt",
    "egamma-egamma",
    "muon-muon",
    "jetmet-jetmet",
]
lumiloss_subdetectors_flags = {
    "PixelPhase1": ["tracker-pixel", "bpix_ready", "fpix_ready"],
    "SiStrip": ["tracker-strip", "tibtid_ready", "tecm_ready", "tecp_ready", "tob_ready"],
    "ECAL": ["ecal-ecal"],
    "ES": ["ecal-es"],
    "HCAL": ["hcal-hcal", "hbhea_ready", "hbheb_ready", "hbhec_ready", "hf_ready", "ho_ready"],
    "CSC": ["csc-csc"],
    "DT": ["dt-dt"],
    "L1T": ["l1t-l1tcalo", "l1t-l1tmu"],
    "HLT": ["hlt-hlt"],
    "Tracking": ["tracker-track"],
    "MuonPOG": ["muon-muon"],
    "JetMET": ["jetmet-jetmet"],
    "EGamma": ["egamma-egamma"],
}

# Acc. luminosity variables
target_acclumi_unit = "/fb"
acc_lumi_year = 2024
acc_lumi_energy_label = "13.6 TeV"
acc_lumi_additional_label_on_plot = "CMS Preliminary Offline Luminosity"  # None if no label

# Eras specific variables (T0)
eras_prefix = "Run2024"
ignore_eras = ["Run2024A"]

# Configure dc3 sensitive variables

In [ ]:
dc3_config.set_keytab_usr(os.getenv("KEYTAB_USR"))
dc3_config.set_keytab_pwd(os.getenv("KEYTAB_PWD"))
dc3_config.set_auth_cert_path(os.getenv("AUTH_CERT"))
dc3_config.set_auth_key_path(os.getenv("AUTH_CERT_KEY"))

# Setup output path if not exists

In [ ]:
os.makedirs(output_path, exist_ok=True)

# Fetch RR and OMS lumisections for all runs

In [ ]:
rra = RunRegistryActions(class_name=rr_class_name, dataset_name=rr_dataset_name)
runs_in_all_cycles = rra.fetch_runs_in_cycles(cycles_list=cycles_list)

In [ ]:
min_run = min(runs_in_all_cycles)
max_run = max(runs_in_all_cycles)
all_datasets = rra.fetch_datasets(min_run=min_run, max_run=max_run)
all_runs = [dataset["run_number"] for dataset in all_datasets]
del all_datasets

In [ ]:
offline_lumis = rra.multi_fetch_rr_oms_joint_lumis(run_list=all_runs)

# Check which runs are not in the DCSOnly Json

In [ ]:
caf = CAF(rr_class_name, kind="dcs")
dcs_json = caf.download(latest=True)
runs_not_in_dcs_json = [run for run in all_runs if str(run) not in dcs_json.keys()]
del caf, dcs_json

# Instantiate the JSON producer

In [ ]:
elegible_runs = [*runs_in_all_cycles, *runs_not_in_dcs_json]
filtered_lumis = [lumi for lumi in offline_lumis if lumi["run_number"] in elegible_runs]
producer = JsonProducer(rr_oms_lumis=filtered_lumis, ignore_hlt_emergency=ignore_hlt_emergency)
del filtered_lumis

# Generate the preJSON

In [ ]:
pjson = producer.generate(prejson_oms_flags)

# Save
base_path = os.path.join(output_path, "jsons")
os.makedirs(base_path, exist_ok=True)
fpath = os.path.join(base_path, "pre.json")
with open(fpath, "w") as f:
    json.dump(pjson, f, ensure_ascii=False, indent=4)

# Generate the goldenJSON

In [ ]:
gjson = producer.generate(goldenjson_oms_flags, goldenjson_rr_flags)

# Save
base_path = os.path.join(output_path, "jsons")
os.makedirs(base_path, exist_ok=True)
fpath = os.path.join(base_path, "golden.json")
with open(fpath, "w") as f:
    json.dump(gjson, f, ensure_ascii=False, indent=4)

# Generate the muonJSON

In [ ]:
mjson = producer.generate(muonjson_oms_flags, muonjson_rr_flags)

# Save
base_path = os.path.join(output_path, "jsons")
os.makedirs(base_path, exist_ok=True)
fpath = os.path.join(base_path, "muon.json")
with open(fpath, "w") as f:
    json.dump(mjson, f, ensure_ascii=False, indent=4)

# Fetch BRIL lumisections for run list

In [ ]:
ba = BrilActions(
    brilws_version=bril_brilws_version,
    unit=bril_unit,
    low_lumi_thr=bril_low_lumi_thr,
    beamstatus=bril_beamstatus,
    amodetag=bril_amodetag,
    normtag=bril_normtag,
)
bril_lumis = ba.fetch_lumis(begin=min_run, end=max_run).get("detailed")
bril_lumis_by_run = ba.agg_by_run(bril_lumis)

# Fetch T0 eras

In [ ]:
t0a = T0Actions()
eras = t0a.eras_history(era=eras_prefix)
eras = [era for era in eras if era["era"] not in ignore_eras]

# Generate lumiloss analysis and acc luminosity plots per era

For each era, filter RR_OMS and BRIL data by the runs contained in the era run range

In [ ]:
eras_statistics = []

for era in eras:
    era_name = era["era"]
    min_run_in_era = era["min_run"]
    max_run_in_era = era["max_run"]

    # All runs (from RR datasets) in this interval
    runs_in_era = [run for run in all_runs if run >= min_run_in_era and run <= max_run_in_era]
    if len(runs_in_era) == 0:
        continue

    # Filter data to era scope
    offline_lumis_in_era = [
        lumi for lumi in offline_lumis if lumi["run_number"] >= min_run_in_era and lumi["run_number"] <= max_run_in_era
    ]
    pjson_in_era = {
        run: lumi_ranges for run, lumi_ranges in pjson.items() if run >= min_run_in_era and run <= max_run_in_era
    }
    gjson_in_era = {
        run: lumi_ranges for run, lumi_ranges in gjson.items() if run >= min_run_in_era and run <= max_run_in_era
    }
    mjson_in_era = {
        run: lumi_ranges for run, lumi_ranges in mjson.items() if run >= min_run_in_era and run <= max_run_in_era
    }
    bril_lumis_in_era = [
        lumi for lumi in bril_lumis if lumi["run_number"] >= min_run_in_era and lumi["run_number"] <= max_run_in_era
    ]
    bril_lumis_by_run_in_era = [
        run for run in bril_lumis_by_run if run["run_number"] >= min_run_in_era and run["run_number"] <= max_run_in_era
    ]

    # It is possible that a run included in an old cycle (from eraB for example) wasn't low lumi at the time
    # and now (with normtag updates) is considered low lumi.
    #
    # This run will appear in both arrays:
    # - runs_in_all_cycles
    # - low_lumi_runs
    #
    # We need to remove this run from the `low_lumi_runs` since it was certified in the past!
    runs_from_cycles_in_era = [run for run in runs_in_all_cycles if run >= min_run_in_era and run <= max_run_in_era]
    low_lumi_runs_in_era = [
        run["run_number"]
        for run in bril_lumis_by_run_in_era
        if run["run_number"] not in runs_from_cycles_in_era and run["has_low_recorded"]
    ]

    # It is possible that the last era contains runs that weren't sent to certification yet
    # because at the time of the latest DC call these runs hadn't all datasets in DQM GUI.
    # We need to ignore this run in lumiloss, but consider in Acc Lumi!
    #
    # To identify these runs:
    # - Shouldn't be in any cycle
    # - Shouldn't be in low lumi list
    # - Shouldn't be in ignore runs list
    # - Shouldn't be in not in DCSOnly list
    runs_not_in_dcs_json_for_era = [
        run for run in runs_in_era if run in runs_not_in_dcs_json and run not in low_lumi_runs_in_era
    ]
    classified_runs = [
        *runs_not_in_dcs_json_for_era,
        *runs_from_cycles_in_era,
        *low_lumi_runs_in_era,
        *ignore_runs_in_any_era,
    ]
    other_runs_in_era = [run for run in runs_in_era if run not in classified_runs]
    ignore_runs_in_any_era.extend(other_runs_in_era)

    print(f"\n{era_name}")
    print("runs_in_era", runs_in_era)
    print("runs_from_cycles_in_era", runs_from_cycles_in_era)
    print("low_lumi_runs_in_era", low_lumi_runs_in_era)
    print("runs_not_in_dcs_json_for_era", runs_not_in_dcs_json_for_era)
    print("ignore_runs_in_any_era", ignore_runs_in_any_era)

    # Check lumiloss
    lumiloss_for_era = LumilossAnalyzer(
        rr_oms_lumis=offline_lumis_in_era,
        bril_lumis=bril_lumis_in_era,
        pre_json=pjson_in_era,
        dc_json=gjson_in_era,
        low_lumi_runs=low_lumi_runs_in_era,
        ignore_runs=ignore_runs_in_any_era,
        bril_unit=bril_unit,
        target_unit=target_lumiloss_unit,
    )
    lumiloss_results = lumiloss_for_era.analyze(
        lumiloss_dcs_flags, lumiloss_subsystems_flags, lumiloss_subdetectors_flags
    )
    lumiloss_results["stats"] = {
        "unit": lumiloss_for_era.final_unit,
        "total_delivered": lumiloss_for_era.total_delivered,
        "total_recorded": lumiloss_for_era.total_recorded,
        "total_low_lumi": lumiloss_for_era.total_low_lumi,
        "total_ignore_runs": lumiloss_for_era.total_ignore_runs,
        "total_not_stable_beams": lumiloss_for_era.total_not_stable_beams,
        "total_not_in_oms_rr": lumiloss_for_era.total_not_in_oms_rr,
        "total_processed": lumiloss_for_era.total_processed,
        "total_loss": lumiloss_for_era.total_loss,
        "total_certified": lumiloss_for_era.total_certified,
        "data_taking_eff": lumiloss_for_era.data_taking_eff,
        "recorded_eff": lumiloss_for_era.recorded_eff,
        "processed_eff": lumiloss_for_era.processed_eff,
    }

    # Setup directory for plots
    era_outpath = output_path + "/eras/" + era_name
    os.makedirs(era_outpath, exist_ok=True)

    # Save jsons
    era_jsons_path = os.path.join(era_outpath, "jsons")
    os.makedirs(era_jsons_path, exist_ok=True)

    fpath = os.path.join(era_jsons_path, "pre.json")
    with open(fpath, "w") as f:
        json.dump(pjson_in_era, f, ensure_ascii=False, indent=4)

    fpath = os.path.join(era_jsons_path, "golden.json")
    with open(fpath, "w") as f:
        json.dump(gjson_in_era, f, ensure_ascii=False, indent=4)

    fpath = os.path.join(era_jsons_path, "muon.json")
    with open(fpath, "w") as f:
        json.dump(mjson_in_era, f, ensure_ascii=False, indent=4)

    # Save lumiloss results
    lumiloss_data_path = os.path.join(era_outpath, "lumiloss/data")
    os.makedirs(lumiloss_data_path, exist_ok=True)

    for key, value in lumiloss_results.items():
        fpath = os.path.join(lumiloss_data_path, f"{key}.json")
        with open(fpath, "w") as f:
            json.dump(value, f)

    # Plot lumiloss
    lumiloss_plots_path = os.path.join(era_outpath, "lumiloss/plots")
    os.makedirs(lumiloss_plots_path, exist_ok=True)

    plots = LumilossPlotter(lumiloss=lumiloss_results, unit=target_lumiloss_unit, output_path=lumiloss_plots_path)
    plots.plot_subsystem_dqmflag_loss(save_pdf=True)
    plots.plot_dcs_loss(save_pdf=True)
    plots.plot_cms_inclusive_loss(save_pdf=True)
    plots.plot_cms_exclusive_loss(save_pdf=True)
    plots.plot_cms_detailed_fraction_exclusive_loss(save_pdf=True)
    plots.plot_inclusive_loss_by_subdetector(save_pdf=True)
    plots.plot_exclusive_loss_by_subdetector(save_pdf=True)
    plots.plot_fraction_of_exclusive_loss_by_subdetector(save_pdf=True)

    # Plot acc luminosity for goldenJSON
    acc_lumi_plots_path = os.path.join(era_outpath, "acc_lumi/golden")
    os.makedirs(acc_lumi_plots_path, exist_ok=True)

    acc_lumi = AccLuminosityAnalyzer(
        dc_json=gjson_in_era,
        bril_lumis=bril_lumis_in_era,
        bril_amodetag=bril_amodetag,
        bril_unit=bril_unit,
        target_unit=target_acclumi_unit,
        year=acc_lumi_year,
        plot_energy_label=acc_lumi_energy_label,
        output_path=acc_lumi_plots_path,
        additional_label_on_plot=acc_lumi_additional_label_on_plot,
    )
    acc_lumi.plot_acc_lumi_by_day(save_pdf=True)
    acc_lumi.plot_acc_lumi_by_week(save_pdf=True)
    del acc_lumi

    # Plot acc luminosity for muonJSON
    acc_lumi_plots_path = os.path.join(era_outpath, "acc_lumi/muon")
    os.makedirs(acc_lumi_plots_path, exist_ok=True)

    acc_lumi = AccLuminosityAnalyzer(
        dc_json=mjson_in_era,
        bril_lumis=bril_lumis_in_era,
        bril_amodetag=bril_amodetag,
        bril_unit=bril_unit,
        target_unit=target_acclumi_unit,
        year=acc_lumi_year,
        plot_energy_label=acc_lumi_energy_label,
        output_path=acc_lumi_plots_path,
        additional_label_on_plot=acc_lumi_additional_label_on_plot,
    )
    acc_lumi.plot_acc_lumi_by_day(save_pdf=True)
    acc_lumi.plot_acc_lumi_by_week(save_pdf=True)
    del acc_lumi, bril_lumis_in_era

    # Save era statistics
    eras_statistics.append(
        {
            "era": era_name,
            "start_run": min_run_in_era,
            "end_run": max_run_in_era,
            "lhc_delivered": lumiloss_for_era.total_delivered,
            "cms_recorded": lumiloss_for_era.total_recorded,
            "total_low_lumi": lumiloss_for_era.total_low_lumi,
            "total_ignore_runs": lumiloss_for_era.total_ignore_runs,
            "total_not_stable_beams": lumiloss_for_era.total_not_stable_beams,
            "total_not_in_oms_rr": lumiloss_for_era.total_not_in_oms_rr,
            "dc_processed": lumiloss_for_era.total_processed,
            "total_loss": lumiloss_for_era.total_loss,
            "dc_certified": lumiloss_for_era.total_certified,
            "processed_eff": lumiloss_for_era.processed_eff,
            "data_taking_eff": lumiloss_for_era.data_taking_eff,
            "recorded_eff": lumiloss_for_era.recorded_eff,
        }
    )

    del (
        pjson_in_era,
        mjson_in_era,
        gjson_in_era,
        offline_lumis_in_era,
        bril_lumis_by_run_in_era,
        low_lumi_runs_in_era,
        lumiloss_for_era,
        plots,
        lumiloss_results,
    )

del offline_lumis

#  Generate dc efficiency per era plot

In [ ]:
eras_eff_plots_path = os.path.join(output_path, "eras")
os.makedirs(eras_eff_plots_path, exist_ok=True)

era_plotter = EraPlotter(eras_statistics, acc_lumi_year, eras_eff_plots_path)
era_plotter.plot_dc_efficiency_by_processed_per_era(save_pdf=True)
era_plotter.plot_dc_efficiency_by_recorded_per_era(save_pdf=True)

del era_plotter, eras_eff_plots_path

# Inspect statistics by era + ALL IN

In [ ]:
print("Min run considered (RR_OMS, BRIL):", min_run)
print("Max run considered (RR_OMS, BRIL):", max_run)

In [ ]:
for stats in eras_statistics:
    print(stats)

In [ ]:
os.makedirs(os.path.join(output_path, "acc_lumi"), exist_ok=True)
all_in_stats_path = os.path.join(output_path, "acc_lumi/stats.json")
with open(all_in_stats_path, "w") as f:
    all_in = {"min_run_rr": min_run, "max_run_rr": max_run, **eras_statistics[-1]}
    json.dump(all_in, f)

# Generate acc luminosity plot for all calls for goldenJSON

In [ ]:
acc_lumi_plots_path = os.path.join(output_path, "acc_lumi/golden")
os.makedirs(acc_lumi_plots_path, exist_ok=True)

acc_lumi = AccLuminosityAnalyzer(
    dc_json=gjson,
    bril_lumis=bril_lumis,
    bril_amodetag=bril_amodetag,
    bril_unit=bril_unit,
    target_unit=target_acclumi_unit,
    year=acc_lumi_year,
    plot_energy_label=acc_lumi_energy_label,
    output_path=acc_lumi_plots_path,
    additional_label_on_plot=acc_lumi_additional_label_on_plot,
)
acc_lumi.plot_acc_lumi_by_day(save_pdf=True)
acc_lumi.plot_acc_lumi_by_week(save_pdf=True)
del acc_lumi

# Generate acc luminosity plot for all calls for muonJSON

In [ ]:
acc_lumi_plots_path = os.path.join(output_path, "acc_lumi/muon")
os.makedirs(acc_lumi_plots_path, exist_ok=True)

acc_lumi = AccLuminosityAnalyzer(
    dc_json=mjson,
    bril_lumis=bril_lumis,
    bril_amodetag=bril_amodetag,
    bril_unit=bril_unit,
    target_unit=target_acclumi_unit,
    year=acc_lumi_year,
    plot_energy_label=acc_lumi_energy_label,
    output_path=acc_lumi_plots_path,
    additional_label_on_plot=acc_lumi_additional_label_on_plot,
)
acc_lumi.plot_acc_lumi_by_day(save_pdf=True)
acc_lumi.plot_acc_lumi_by_week(save_pdf=True)
del acc_lumi, bril_lumis, mjson, pjson, gjson